# Title: Reproducibility_Manager (Seed_Everything)
## Description: Python, Numpy, PyTorch, CUDA 등 딥러닝 파이프라인에 관여하는 모든 난수 생성 시드(Seed)를 고정하여 실험의 재현성을 보장하는 설정 모듈.
## Input: 
 - seed (int): 고정할 시드 번호 (Default: 42)
## Output: 
 - None (Global Environment Setup)
## Check Point: 
 - `cudnn.deterministic = True` 설정은 연산 속도를 약간 저하시킬 수 있으나, 실험 단계에서는 재현성이 우선됨.
 - Hash Seed 고정(`os.environ`)은 딕셔너리/Set 순서 유지에 필수적.

In [ ]:
import os
import random
import numpy as np
import torch

def seed_everything(seed=42):
    """
    모든 랜덤 시드를 고정하여 실험의 재현성(Reproducibility)을 확보하는 함수
    """
    # 1. 파이썬 내장 랜덤 모듈 고정
    random.seed(seed) 
    
    # 2. 해시 함수 고정 (중요)
    # 딕셔너리나 Set의 내부 순서가 실행 때마다 바뀌는 것을 방지
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    # 3. Numpy 난수 고정 (데이터 분할, 전처리 등에서 필수)
    np.random.seed(seed)
    
    # 4. PyTorch CPU/GPU 난수 고정 (가중치 초기화 등)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) # 멀티 GPU 사용 시
    
    # 5. CUDNN(CUDA Deep Neural Network library) 결정론적 모드 설정
    # True로 하면: 항상 같은 알고리즘을 써서 결과가 완벽히 동일 (재현성 최우선)
    torch.backends.cudnn.deterministic = True

    # True로 하면: 하드웨어 상황에 맞춰 제일 빠른 알고리즘을 자동 선택 (속도 최우선)
    # 학습용이니 False
    torch.backends.cudnn.benchmark = False
    
    print(f'🔒 Seed set to {seed}. Reproducibility secured.')

# 실행 예시
# seed_everything(42)

## How to Use
1. **맨 처음에 실행**: 코드의 가장 윗부분(임포트 직후)에서 실행해야 효과가 있습니다.
    ```python
    import ...
    
    CFG = {'SEED': 41}
    seed_everything(CFG['SEED'])
    ```

## Troubleshooting
- **속도가 너무 느릴 때**: 모델 구조 탐색이 끝나고, 단순히 학습을 오래 돌려야 할 때는 `torch.backends.cudnn.benchmark = True`로 변경하여 속도를 높일 수 있습니다. (단, 이때부터는 결과가 미세하게 달라질 수 있음)
- **일부 라이브러리**: Scikit-learn이나 Albumentations 등 별도의 시드를 요구하는 라이브러리는 함수 호출 시 `random_state=seed` 인자를 명시적으로 넣어주어야 완벽하게 고정됩니다.